## 데이터

In [1]:
pwd

'C:\\Users\\stand0\\Desktop\\2025\\공공빅데이터분석'

In [2]:
import re
import os
import glob
import requests
import pandas as pd

## 데이터 인코딩 변경해서 저장

In [3]:
# c = pd.read_csv('./food_data/법정동.csv', encoding='euc-kr') # utf-8
# c = c[c['시도명']=='대전광역시']
# c.to_csv('./food_data/d_pop.csv', encoding='utf-8-sig', index=False)

## 데이터 확인
### 인구 데이터

### 전국 인구 데이터

In [4]:
df_pop = pd.read_csv('./food_data/a_pop.csv', encoding='utf-8-sig') # utf-8
df_pop.columns

Index(['법정동코드', '기준연월', '시도명', '시군구명', '읍면동명', '리명', '계', '남자', '여자', '0세남자',
       ...
       '101세여자', '102세여자', '103세여자', '104세여자', '105세여자', '106세여자', '107세여자',
       '108세여자', '109세여자', '110세이상 여자'],
      dtype='object', length=231)

In [5]:
# 1) 불필요 컬럼 삭제
df_pop = df_pop.drop(columns=['기준연월', '계'])

# 2) 남자/여자 나이별 컬럼만 골라내기
male_cols = [col for col in df_pop.columns if re.match(r'\d+세(?:이상)?\s*남자', col)]
female_cols = [col for col in df_pop.columns if re.match(r'\d+세(?:이상)?\s*여자', col)]

In [6]:
# 3) 0~100세를 5세 단위로 자를 bin 정의
bins = [(i, i+4) for i in range(0, 100, 5)] + [(100, 100)] # 0부터 100까지

In [7]:
# 4) 결과를 담을 DataFrame 생성 (원본에서 보존할 키컬럼)
key_cols = ['시도명', '시군구명', '읍면동명']
new_df = df_pop[key_cols].copy()

In [8]:
# 5) 각 bin마다 남/여 합계 계산
for start, end in bins:
    if start == end:
        # 100세 단일 연령
        mcols = [f"{start}세남자"] if f"{start}세남자" in df_pop.columns else []
        fcols = [f"{start}세여자"] if f"{start}세여자" in df_pop.columns else []
        label = f"{start}세"
    else:
        # 예: start=0,end=4 → ages=[0,1,2,3,4]
        ages = list(range(start, end+1))
        mcols = [f"{age}세남자" for age in ages if f"{age}세남자" in df_pop.columns]
        fcols = [f"{age}세여자" for age in ages if f"{age}세여자" in df_pop.columns]
        label = f"{start}-{end}세"

    # 합산한 결과를 new_df에 추가
    if mcols:
        new_df[f"{label}_남자"] = df_pop[mcols].sum(axis=1)
    else:
        new_df[f"{label}_남자"] = 0

    if fcols:
        new_df[f"{label}_여자"] = df_pop[fcols].sum(axis=1)
    else:
        new_df[f"{label}_여자"] = 0

# 6) 101세 이상
# 컬럼명이 '101세남자','102세남자',…,'110세이상 남자' 처럼 되어 있으므로, 101 이상인 남/여 컬럼 전부를 모아서 합산
male_over = [
    col for col in df_pop.columns
    if re.match(r'(\d+)세(?:이상)?\s*남자', col) and int(re.match(r'(\d+)', col).group(1)) >= 101
]
female_over = [
    col for col in df_pop.columns
    if re.match(r'(\d+)세(?:이상)?\s*여자', col) and int(re.match(r'(\d+)', col).group(1)) >= 101
]

new_df["101세이상_남자"] = df_pop[male_over].sum(axis=1) if male_over else 0
new_df["101세이상_여자"] = df_pop[female_over].sum(axis=1) if female_over else 0

# 기존 컬럼과 교체
df_pop = new_df
df_pop.head()

,시도명,시군구명,읍면동명,0-4세_남자,0-4세_여자,5-9세_남자,5-9세_여자,10-14세_남자,10-14세_여자,15-19세_남자,...,85-89세_남자,85-89세_여자,90-94세_남자,90-94세_여자,95-99세_남자,95-99세_여자,100세_남자,100세_여자,101세이상_남자,101세이상_여자
0,서울특별시,종로구,청운동,19,19,53,30,76,59,86,...,22,32,4,9,2,1,0,0,0,1
1,서울특별시,종로구,신교동,11,8,18,11,31,37,35,...,13,18,1,5,0,4,0,0,0,0
2,서울특별시,종로구,궁정동,1,1,5,5,10,11,8,...,1,1,0,2,0,0,0,0,2,0
3,서울특별시,종로구,효자동,8,3,10,11,9,7,9,...,2,1,1,5,0,1,0,0,0,0
4,서울특별시,종로구,창성동,3,6,5,3,8,7,6,...,3,2,2,3,1,1,0,0,0,0


In [9]:
# df_pop.to_csv('./food_data/a_pop_cleaned.csv', encoding='utf-8-sig', index=False)

In [10]:
# 1) 인구 수치 컬럼만 골라내기 (문자형 key 컬럼 제외)
pop_cols = df_pop.columns.drop(['시도명', '시군구명', '읍면동명'])

# 2) 시군구별로 그룹화해서 합산
df_sgg = (
    df_pop
    .groupby(['시도명', '시군구명'])[pop_cols]
    .sum()                    # 시군구별 연령·성별 합계 DataFrame
    .sum(axis=1)              # 각 시군구별로 열(구간) 전부 합산 → Series
    .reset_index(name='총인구')
)

# 결과 확인
df_sgg

,시도명,시군구명,총인구
0,강원특별자치도,강릉시,207069
1,강원특별자치도,고성군,26824
2,강원특별자치도,동해시,86931
3,강원특별자치도,삼척시,61428
4,강원특별자치도,속초시,80141
...,...,...,...
247,충청북도,청주시 상당구,197250
248,충청북도,청주시 서원구,180794
249,충청북도,청주시 청원구,188866
250,충청북도,청주시 흥덕구,288358


### 대전 인구 데이터

In [11]:
df_pop = pd.read_csv('./food_data/d_pop.csv', encoding='utf-8-sig') # utf-8
df_pop.columns

Index(['법정동코드', '기준연월', '시도명', '시군구명', '읍면동명', '리명', '계', '남자', '여자', '0세남자',
       ...
       '101세여자', '102세여자', '103세여자', '104세여자', '105세여자', '106세여자', '107세여자',
       '108세여자', '109세여자', '110세이상 여자'],
      dtype='object', length=231)

In [12]:
# # 1) 불필요 컬럼 삭제
df_pop = df_pop.drop(columns=['기준연월', '시도명', '계'])

# 2) 남자/여자 나이별 컬럼만 골라내기
male_cols = [col for col in df_pop.columns if re.match(r'\d+세(?:이상)?\s*남자', col)]
female_cols = [col for col in df_pop.columns if re.match(r'\d+세(?:이상)?\s*여자', col)]

In [13]:
# 3) 0~100세를 5세 단위로 자를 bin 정의
bins = [(i, i+4) for i in range(0, 100, 5)] + [(100, 100)] # 0부터 100까지

In [14]:
# 4) 결과를 담을 DataFrame 생성 (원본에서 보존할 키컬럼)
key_cols = ['법정동코드', '시군구명', '읍면동명', '리명']
new_df = df_pop[key_cols].copy()

In [15]:
# 5) 각 bin마다 남/여 합계 계산
for start, end in bins:
    if start == end:
        # 100세 단일 연령
        mcols = [f"{start}세남자"] if f"{start}세남자" in df_pop.columns else []
        fcols = [f"{start}세여자"] if f"{start}세여자" in df_pop.columns else []
        label = f"{start}세"
    else:
        # 예: start=0,end=4 → ages=[0,1,2,3,4]
        ages = list(range(start, end+1))
        mcols = [f"{age}세남자" for age in ages if f"{age}세남자" in df_pop.columns]
        fcols = [f"{age}세여자" for age in ages if f"{age}세여자" in df_pop.columns]
        label = f"{start}-{end}세"

    # 합산한 결과를 new_df에 추가
    if mcols:
        new_df[f"{label}_남자"] = df_pop[mcols].sum(axis=1)
    else:
        new_df[f"{label}_남자"] = 0

    if fcols:
        new_df[f"{label}_여자"] = df_pop[fcols].sum(axis=1)
    else:
        new_df[f"{label}_여자"] = 0

# 6) 101세 이상
# 컬럼명이 '101세남자','102세남자',…,'110세이상 남자' 처럼 되어 있으므로, 101 이상인 남/여 컬럼 전부를 모아서 합산
male_over = [
    col for col in df_pop.columns
    if re.match(r'(\d+)세(?:이상)?\s*남자', col) and int(re.match(r'(\d+)', col).group(1)) >= 101
]
female_over = [
    col for col in df_pop.columns
    if re.match(r'(\d+)세(?:이상)?\s*여자', col) and int(re.match(r'(\d+)', col).group(1)) >= 101
]

new_df["101세이상_남자"] = df_pop[male_over].sum(axis=1) if male_over else 0
new_df["101세이상_여자"] = df_pop[female_over].sum(axis=1) if female_over else 0

# 기존 컬럼과 교체
df_pop = new_df
df_pop

,법정동코드,시군구명,읍면동명,리명,0-4세_남자,0-4세_여자,5-9세_남자,5-9세_여자,10-14세_남자,10-14세_여자,...,85-89세_남자,85-89세_여자,90-94세_남자,90-94세_여자,95-99세_남자,95-99세_여자,100세_남자,100세_여자,101세이상_남자,101세이상_여자
0,3011010100,동구,원동,NaN,1,2,2,3,3,6,...,3,10,1,1,0,2,0,0,0,0
1,3011010200,동구,인동,NaN,66,64,68,82,119,105,...,42,89,14,28,0,16,0,0,2,1
2,3011010300,동구,효동,NaN,11,14,25,30,40,30,...,31,35,8,27,2,5,0,0,0,0
3,3011010400,동구,천동,NaN,542,452,378,345,424,418,...,32,77,4,18,0,9,0,1,0,1
4,3011010500,동구,가오동,NaN,130,119,250,256,430,420,...,38,91,5,31,2,12,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,3023012000,대덕구,이현동,NaN,1,1,0,0,1,1,...,1,2,0,0,0,1,0,0,0,0
170,3023012100,대덕구,갈전동,NaN,0,0,0,1,0,0,...,0,1,0,1,0,2,0,0,0,0
171,3023012400,대덕구,삼정동,NaN,0,0,2,1,0,2,...,1,5,1,0,0,0,0,0,0,0
172,3023012500,대덕구,미호동,NaN,2,2,0,1,4,3,...,4,4,0,2,0,1,0,0,0,1


In [16]:
# 1) 인구 수치 컬럼만 골라내기 (문자형 key 컬럼 제외)
pop_cols = df_pop.columns.drop(['법정동코드', '시군구명', '읍면동명', '리명'])

# 2) 시군구별로 그룹화해서 합산
df_sgg = (
    df_pop
    .groupby(['시군구명'])[pop_cols]
    .sum()                    # 시군구별 연령·성별 합계 DataFrame
    .sum(axis=1)              # 각 시군구별로 열(구간) 전부 합산 → Series
    .reset_index(name='총인구')
)

# 결과 확인
df_sgg

,시군구명,총인구
0,대덕구,166098
1,동구,220067
2,서구,461085
3,유성구,367488
4,중구,224869


In [17]:
# 1) 인구 수치 컬럼만 골라내기 (문자형 key 컬럼 제외)
pop_cols = df_pop.columns.drop(['법정동코드', '시군구명', '읍면동명', '리명'])

# 2) 시군구별로 그룹화해서 합산
df_sgg = (
    df_pop
    .groupby(['시군구명', '읍면동명'])[pop_cols]
    .sum()                    # 시군구별 연령·성별 합계 DataFrame
    .sum(axis=1)              # 각 시군구별로 열(구간) 전부 합산 → Series
    .reset_index(name='총인구')
)

# 결과 확인
df_sgg

,시군구명,읍면동명,총인구
0,대덕구,갈전동,41
1,대덕구,대화동,5391
2,대덕구,덕암동,8279
3,대덕구,목상동,5511
4,대덕구,문평동,14
...,...,...,...
169,중구,정생동,199
170,중구,중촌동,14477
171,중구,침산동,134
172,중구,태평동,35874


In [18]:
df_pop.to_csv('./food_data/d_pop_cleaned.csv', encoding='utf-8-sig', index=False) ## df_sgg랑은 다름

### 전국 소매점 데이터

In [19]:
# # 1) 대상 소분류 리스트 정의
# target_stores = [
#     '곡물/곡분 소매업',
#     '정육점',
#     '건어물/젓갈 소매업',
#     '수산물 소매업',
#     '채소/과일 소매업',
#     '반찬/식료품 소매업'
# ]

# district_rows = []

# # 3) 폴더 내 모든 CSV 파일 순회
# for filepath in glob.glob('./food_data/소상공인시장진흥공단_상권/*202503.csv'):
#     # 파일명에서 지역명 추출 (예: ..._강원_202503.csv → '강원')
#     region = os.path.basename(filepath).split('_')[-2]
#     df = pd.read_csv(filepath, encoding='utf-8-sig', low_memory=False)
    
#     # 컬럼명 통일
#     df = df.rename(columns={
#         '상권업종중분류명': '중분류',
#         '상권업종소분류명': '소분류'
#     })

#     # 시군구별 그룹
#     for sgg, grp in df.groupby('시군구명'):
        
#         district_rows.append({
#             '지역': region,
#             '시군구': sgg,
#             '종합 소매업': grp['소분류'].isin(['슈퍼마켓','그외 기타 종합 소매업']).sum(),
#             '전문 식료품 소매업': grp['소분류'].isin(target_stores).sum(),
#             '편의점': (grp['소분류']=='편의점').sum()
#         })

# # 4) 시군구 단위 DataFrame
# df_district = pd.DataFrame(district_rows)

# # 5) 지역별(도‧광역시) 평균 내기
# df_region = (
#     df_district
#     .groupby('지역')[['종합 소매업','전문 식료품 소매업','편의점']]
#     .mean()
#     .reset_index()
# )

# # 5) 전국(모든 시군구) 평균 추가
# nation_avg = df_district[['종합 소매업','전문 식료품 소매업','편의점']].mean()
# nation_avg_row = pd.DataFrame([{
#     '지역': '전국 평균',
#     '종합 소매업': nation_avg['종합 소매업'],
#     '전문 식료품 소매업': nation_avg['전문 식료품 소매업'],
#     '편의점': nation_avg['편의점']
# }])

# df_final = pd.concat([df_region, nation_avg_row], ignore_index=True)

# df_final

In [20]:
# df_final.to_csv('./food_data/a_market_cleaned.csv', encoding='utf-8-sig', index=False) 

In [21]:
df_market_a = pd.read_csv('./food_data/a_market_cleaned.csv', encoding='utf-8-sig')

In [22]:
df_market_a

,지역,종합 소매업,전문 식료품 소매업,편의점
0,강원,159.944444,249.111111,135.777778
1,경기,303.409091,336.090909,350.204545
2,경남,209.090909,208.227273,156.272727
3,경북,192.086957,192.304348,123.000000
4,광주,340.400000,364.000000,282.000000
5,대구,259.444444,262.888889,223.555556
6,대전,309.000000,379.400000,291.800000
7,부산,223.687500,210.500000,175.125000
8,서울,321.400000,405.120000,395.280000
9,세종,294.000000,265.000000,299.000000


### 대전 소매점 데이터

In [23]:
df_market = pd.read_csv('./food_data/d_market.csv', encoding='utf-8-sig') # utf-8

# 컬럼명 변경
df_market = df_market.rename(columns={'상권업종중분류명' : '중분류', '상권업종소분류명' : '소분류'})

In [24]:
# 시군구+법정동 컬럼 추가
df_market['시군구+법정동'] = df_market['시군구명'] + ' ' + df_market['법정동명']

# 전문 식료품점
target_stores = [
    '곡물/곡분 소매업',
    '정육점',
    '건어물/젓갈 소매업',
    '수산물 소매업',
    '채소/과일 소매업',
    '반찬/식료품 소매업'
]

# 동(법정동) 단위로 개수 집계
df_neighborhood = (
    df_market
    .groupby(['시군구명', '시군구+법정동']) 
    .apply(lambda grp: pd.Series({
        '종합 소매업': grp['소분류']
                         .isin(['슈퍼마켓', '그외 기타 종합 소매업'])
                         .sum(),
        '전문 식료품 소매업': grp['소분류']
                             .isin(target_stores)
                             .sum(),
        '편의점': (grp['소분류'] == '편의점').sum()
    }), include_groups=False)
    .reset_index()
)

# 시군구별 평균 행
df_sgg_avg = (
    df_neighborhood
    .groupby('시군구명')[['종합 소매업','전문 식료품 소매업','편의점']]
    .mean()
    .reset_index()
)
# 평균용 이름 붙여주기
df_sgg_avg['시군구+법정동'] = '평균'

# 최종 결합
df_final = pd.concat([df_neighborhood, df_sgg_avg], ignore_index=True)
df_final

,시군구명,시군구+법정동,종합 소매업,전문 식료품 소매업,편의점
0,대덕구,대덕구 갈전동,0.000000,0.000000,0.000000
1,대덕구,대덕구 대화동,19.000000,9.000000,6.000000
2,대덕구,대덕구 덕암동,9.000000,6.000000,12.000000
3,대덕구,대덕구 목상동,4.000000,5.000000,10.000000
4,대덕구,대덕구 문평동,0.000000,0.000000,5.000000
...,...,...,...,...,...
171,대덕구,평균,9.750000,8.583333,7.583333
172,동구,평균,7.333333,9.761905,5.047619
173,서구,평균,15.115385,22.192308,18.307692
174,유성구,평균,5.867925,6.037736,6.962264


In [25]:
# df_final.to_csv('./food_data/d_market_cleaned.csv', encoding='utf-8-sig', index=False)
df_market_d = pd.read_csv('./food_data/d_market_cleaned.csv', encoding='utf-8-sig')
df_market_d

,시군구명,시군구+법정동,종합 소매업,전문 식료품 소매업,편의점
0,대덕구,대덕구 갈전동,0.000000,0.000000,0.000000
1,대덕구,대덕구 대화동,19.000000,9.000000,6.000000
2,대덕구,대덕구 덕암동,9.000000,6.000000,12.000000
3,대덕구,대덕구 목상동,4.000000,5.000000,10.000000
4,대덕구,대덕구 문평동,0.000000,0.000000,5.000000
...,...,...,...,...,...
171,대덕구,평균,9.750000,8.583333,7.583333
172,동구,평균,7.333333,9.761905,5.047619
173,서구,평균,15.115385,22.192308,18.307692
174,유성구,평균,5.867925,6.037736,6.962264


---

# 지도 데이터 면적 구하기

In [20]:
import geopandas as gpd

# Load GeoDataFrame
shp_path = "./food_data/LSMD_ADM_SECT_UMD_30_202505.shp"
gdf = gpd.read_file(shp_path)

gdf.info(), gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   EMD_CD      177 non-null    object  
 1   COL_ADM_SE  177 non-null    object  
 2   EMD_NM      177 non-null    object  
 3   SGG_OID     177 non-null    int32   
 4   geometry    177 non-null    geometry
dtypes: geometry(1), int32(1), object(3)
memory usage: 6.3+ KB


(None,
      EMD_CD COL_ADM_SE  EMD_NM  SGG_OID  \
 0  30110107      30110  ÆÇ¾Ïµ¿     3363   
 1  30110126      30110  È¿Æòµ¿     3346   
 2  30110119      30110    Á¤µ¿     2705   
 3  30110102      30110    ÀÎµ¿     3351   
 4  30110132      30110  »ç¼ºµ¿     3345   
 
                                             geometry  
 0  POLYGON ((240138.64 413718.994, 240139.185 413...  
 1  POLYGON ((240495.584 420426.648, 240511.773 42...  
 2  POLYGON ((238468.622 415228.476, 238473.435 41...  
 3  POLYGON ((238843.284 414267.582, 238852.143 41...  
 4  POLYGON ((245382.677 421506.914, 245386.375 42...  )

In [21]:
# GeoPandas의 from_file 대신 fiona로 직접 레이어 확인
import fiona

# Check the layers and schema
with fiona.open(shp_path, 'r') as src:
    meta = src.meta
    schema = src.schema
    crs = src.crs
    sample = next(iter(src))

(meta, schema, crs, sample)

({'driver': 'ESRI Shapefile',
  'schema': {'properties': {'EMD_CD': 'str:8',
    'COL_ADM_SE': 'str:5',
    'EMD_NM': 'str:100',
    'SGG_OID': 'int32:9'},
   'geometry': 'Polygon'},
  'crs': CRS.from_wkt('PROJCS["Korea 2000 / Central Belt 2010",GEOGCS["Korea 2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["central_meridian",127],PARAMETER["latitude_of_origin",38],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["m",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5186"]]'),
  'crs_wkt': 'PROJCS["Korea 2000 / Central Belt 2010",GEOGCS["Korea 2000",DATUM["Geocentric_datum_of_Korea",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG"

In [22]:
# 다시 GeoDataFrame으로 shapefile을 불러오기 (좌표계 설정 명시)
gdf = gpd.read_file(shp_path)

# EPSG:5186 -> 면적 단위는 제곱미터. 나중에 km²로 변환 가능.
# 대전광역시는 법정동 코드 앞자리 30으로 시작함 (ex. 30110xxx)
daejeon_gdf = gdf[gdf["EMD_CD"].str.startswith("30")].copy()

# 면적(m^2) 계산 후 km^2 단위로 변환
daejeon_gdf["area_km2"] = daejeon_gdf["geometry"].area / 1_000_000

# 필요한 열만 추출
daejeon_area_df = daejeon_gdf[["EMD_CD", "EMD_NM", "area_km2"]].sort_values("EMD_CD").reset_index(drop=True)

daejeon_area_df

,EMD_CD,EMD_NM,area_km2
0,30110101,¿øµ¿,0.187632
1,30110102,ÀÎµ¿,0.418281
2,30110103,È¿µ¿,0.174106
3,30110104,Ãµµ¿,0.742605
4,30110105,°¡¿Àµ¿,1.512375
...,...,...,...
172,30230122,ºÎ¼öµ¿,0.966540
173,30230123,È²È£µ¿,2.667417
174,30230124,»ïÁ¤µ¿,3.054537
175,30230125,¹ÌÈ£µ¿,4.307916


In [24]:
# daejeon_area_df.to_csv('./food_data/d_area.csv', encoding='cp949', index=False)
daejeon_area_df.to_csv("./food_data/d_area.csv", index=False, encoding="utf-8-sig")

### 인코딩 감지

In [ ]:
# !pip install chardet

In [ ]:
# import chardet

# with open('/content/drive/MyDrive/1_public/markets_data/markets_st_2024.csv', 'rb') as f:
#     rawdata = f.read(10000)
#     result = chardet.detect(rawdata)
#     print(result)

---